In [1]:
import os

In [2]:
#function to compute XOR for two hexadecimals of different lengths
def hexxor(a, b): 
    if len(a) > len(b):
        c = hex(int(a[:len(b)],16)^int(b,16))[2:]
    else:
        c = hex(int(a,16)^int(b[:len(a)],16))[2:]
    if len(c)%2:
        c = "".join(['0',c])
    return c

In [3]:
# Open the file in read mode
with open("sentences encrypted.txt", "r", encoding="utf-8") as file:
    # Read all lines from the file and store them in a list
    lines_encrypted = file.readlines()
# Strip whitespace and newline characters from each line
lines_encrypted = [line.strip() for line in lines_encrypted]
with open("flag encrypted.txt", "r", encoding="utf-8") as file:
    # Read flag from the file flag.txt and store them in flag
    flag_encrypted = file.read()

In [4]:
cc = lines_encrypted.copy() #To make any changes without affecting the correct lines
#To determine the length of the lines in order to create a key matching the longest line
ccc = [0]*len(cc) 
for i in range(len(cc)) : 
    ccc[i]=len(cc[i])
nkey = [0] * max(ccc)
#To determine the part of the key to work on, the text must include at least one letter, one space, and another letter different from the first.
#This requires at least 3 sentences with lengths equal to or greater than the length we are working on.
while (sum(1 for i in ccc if i==max(ccc)) <= 2) : 
    r=ccc.index(max(ccc))
    ccc[r]=max(ccc)-1
#Since each character is 1 byte (8 bits), and the length in Hexadecimal is represented as 4 bits per digit, we divide the number of Hex digits by 2
#to determine the number of characters or bytes to process
for a in range(int(max(ccc) / 2)):
    ci = '00' #To place the character from the encrypted line that corresponds to an unencrypted character which is a space (espace)
    l = 0 #Just to check if we found the character in the key. If l=1, it means we found the key character; if l=0, it means we didn’t
    for i in range(len(cc) - 1):
        for j in range(i + 1, len(cc)): #We use 2 loops to perform XOR between all the characters of the lines with each other
            if len(cc[i]) <= a * 2 or len(cc[j]) <= a * 2: 
                continue #To avoid a list index out-of-bounds error
            if int(hexxor(cc[i][a * 2], cc[j][a * 2])) in [4, 5, 6, 7]: #If the XOR result is 4, 5, 6, or 7, it means one is a space (espace) and the other is a letter (lettre)
                for k in range(len(cc)): #To determine exactly which one is the space (espace)
                    if k == i or k == j or len(cc[k]) <= a * 2: 
                        continue #To avoid a list index out-of-bounds error and performing xor again between i and j
                    if int(hexxor(cc[i][a * 2], cc[k][a * 2])) not in [4, 5, 6, 7] and hexxor(cc[i][a * 2:a * 2+2], cc[k][a * 2:a * 2+2]) != '00' :
                        ci = cc[j][a * 2:a * 2 + 2]
                        break #If the XOR result is different from 4, 5, 6, or 7, it means the operation was between two letters (lettres) or two spaces (espaces).
                             #If the result is not 00, it means the operation was between two different letters. Therefore, the other character is a space (espace)
                    else:
                        ci = cc[i][a * 2:a * 2 + 2] #If all XOR results with the character at index i are also 4, 5, or 6, or 7, it means it is is an espace
                l = 1 #because the XOR result is 4, 5, 6, or 7, it means one is a space (espace) and the other is a letter (lettre)
                break
        if l == 1: #If l=1, it means m is a space (espace). Therefore, the key is the result of XOR between c and the m key we defined
            nkey[a * 2:a * 2 + 2] = hexxor(ci, '20') 
            break
    if l == 0: #If we don't get any result, we change c to 21 and key to 00 to get a "!" marker wherever we haven't found the key
        for i in range(len(cc)): #This loop is for the characters that are smaller than the part of the key we are working on
            if len(cc[i]) <= a * 2:
                continue #To avoid a list index out-of-bounds error
            cc[i] = list(cc[i]) #to change cc from str to list to can perform changes
            cc[i][a * 2:a * 2 + 2] = '21' #to change c to 21
            cc[i] = ''.join(cc[i]) #to change cc from list to str to can perform hexxor 
        nkey[a * 2:a * 2 + 2] = '00' #to change k to 00
for i in range(len(cc)): #This loop is for the characters that are larger than the part of the key we worked on
    if len(cc[i]) > max(ccc) :
        cc[i] = list(cc[i])
        for j in range(max(ccc),len(cc[i]),2) :
            cc[i][j:j+2] = '21'
            nkey[j:j+2] = '00'
        cc[i] = ''.join(cc[i])
nkey = ''.join(nkey)
print(nkey) 

00000000a6b33c006d2565f212e0009b34267700270f8c2f01799eac000000e187001f0b5484fe00c46200000000


In [5]:
#to display the characters for which we were able to find the key to decrypt, 
#while the characters we couldn't decrypt will appear with a "!" symbol
for i in range(len(cc)) : 
    print(bytes.fromhex(hexxor(nkey,cc[i])).decode('utf-8'))

!!!! ar! fun g!mes to test h!!!in! skil!s
!!!! ha!e task! like web mal!!!e !wn cr!pt!
!!!! he!p you !earn how to h!!! a!d sta! s!!!
!!!! al!ne or !n a team to s!!!e !ool p!ob!!!!
!!!! ar! for b!th new and ex!!!t !ecuri!y !!!!
!!!! a !TF onl!ne and learn !!!m !nywhe!e !!!!
!!!!e t!sks wi! points and h!!! f!n in !TF!


In [6]:
lines_encrypted.append(flag_encrypted) #To add the encrypted flag to the other encrypted lines
cc.append(flag_encrypted)
#function to determine the key if we deduced one of the hidden characters in the previous step
#l takes the value of the line number where we identified the character, and the line numbers start from 1
#i is the character index
#c is the character itself
#The other variables were explained earlier
def pe(l,i,c,nkey,cc,lines_encrypted):
    nkey[2*i-2:2*i] = hexxor(lines_encrypted[l-1][2*i-2:2*i], format(ord(c), "x")) #Calculate the XOR between the assumed character from the sentences and the byte in the encrypted line to find the key
    for j in range(len(cc)-1) : #To replace the correct encrypted byte with the byte we previously set to 21 as a "!" marker, so that when we XOR the key with the encrypted byte, the correct decrypted character appears
        cc[j] = list(cc[j])
        cc[j][2*i-2:2*i] = lines_encrypted[j][2*i-2:2*i]
        cc[j] = ''.join(cc[j])
    nkey = ''.join(nkey)
    print(nkey)
    for i in range(len(cc)-1) : #To display the sentences with the characters we successfully decrypted
        print(bytes.fromhex(hexxor(nkey,cc[i])).decode('utf-8'))

In [7]:
#We know the format of the flag "DEFENSYS{*}", so we recognize the first four characters that contain the ! symbol. Therefore
#we will use the pe function to find the first 4 bytes of the key and decrypt the sentences
#We will continue with this approach using other words like "g!mes" which clarify "games"...
#until we find the full key and decrypt all the sentences and the flag
nkey = list(nkey)
pe(8,1,"D",nkey,cc,lines_encrypted)

c5000000a6b33c006d2565f212e0009b34267700270f8c2f01799eac000000e187001f0b5484fe00c46200000000
C!!! ar! fun g!mes to test h!!!in! skil!s
T!!! ha!e task! like web mal!!!e !wn cr!pt!
C!!! he!p you !earn how to h!!! a!d sta! s!!!
W!!! al!ne or !n a team to s!!!e !ool p!ob!!!!
C!!! ar! for b!th new and ex!!!t !ecuri!y !!!!
J!!! a !TF onl!ne and learn !!!m !nywhe!e !!!!
S!!!e t!sks wi! points and h!!! f!n in !TF!


In [8]:
nkey = list(nkey)
pe(8,2,"E",nkey,cc,lines_encrypted)

c5ac0000a6b33c006d2565f212e0009b34267700270f8c2f01799eac000000e187001f0b5484fe00c46200000000
CT!! ar! fun g!mes to test h!!!in! skil!s
Th!! ha!e task! like web mal!!!e !wn cr!pt!
CT!! he!p you !earn how to h!!! a!d sta! s!!!
Wo!! al!ne or !n a team to s!!!e !ool p!ob!!!!
CT!! ar! for b!th new and ex!!!t !ecuri!y !!!!
Jo!! a !TF onl!ne and learn !!!m !nywhe!e !!!!
So!!e t!sks wi! points and h!!! f!n in !TF!


In [9]:
nkey = list(nkey)
pe(8,3,"F",nkey,cc,lines_encrypted)

c5ac0400a6b33c006d2565f212e0009b34267700270f8c2f01799eac000000e187001f0b5484fe00c46200000000
CTF! ar! fun g!mes to test h!!!in! skil!s
The! ha!e task! like web mal!!!e !wn cr!pt!
CTF! he!p you !earn how to h!!! a!d sta! s!!!
Wor! al!ne or !n a team to s!!!e !ool p!ob!!!!
CTF! ar! for b!th new and ex!!!t !ecuri!y !!!!
Joi! a !TF onl!ne and learn !!!m !nywhe!e !!!!
Sol!e t!sks wi! points and h!!! f!n in !TF!


In [10]:
nkey = list(nkey)
pe(8,4,"E",nkey,cc,lines_encrypted)

c5ac0412a6b33c006d2565f212e0009b34267700270f8c2f01799eac000000e187001f0b5484fe00c46200000000
CTFs ar! fun g!mes to test h!!!in! skil!s
They ha!e task! like web mal!!!e !wn cr!pt!
CTFs he!p you !earn how to h!!! a!d sta! s!!!
Work al!ne or !n a team to s!!!e !ool p!ob!!!!
CTFs ar! for b!th new and ex!!!t !ecuri!y !!!!
Join a !TF onl!ne and learn !!!m !nywhe!e !!!!
Solve t!sks wi! points and h!!! f!n in !TF!


In [11]:
nkey = list(nkey)
pe(7,8,"a",nkey,cc,lines_encrypted)

c5ac0412a6b33cbc6d2565f212e0009b34267700270f8c2f01799eac000000e187001f0b5484fe00c46200000000
CTFs are fun g!mes to test h!!!in! skil!s
They have task! like web mal!!!e !wn cr!pt!
CTFs help you !earn how to h!!! a!d sta! s!!!
Work alone or !n a team to s!!!e !ool p!ob!!!!
CTFs are for b!th new and ex!!!t !ecuri!y !!!!
Join a CTF onl!ne and learn !!!m !nywhe!e !!!!
Solve tasks wi! points and h!!! f!n in !TF!


In [12]:
nkey = list(nkey)
pe(1,15,"a",nkey,cc,lines_encrypted)

c5ac0412a6b33cbc6d2565f212e0969b34267700270f8c2f01799eac000000e187001f0b5484fe00c46200000000
CTFs are fun games to test h!!!in! skil!s
They have tasks like web mal!!!e !wn cr!pt!
CTFs help you learn how to h!!! a!d sta! s!!!
Work alone or in a team to s!!!e !ool p!ob!!!!
CTFs are for both new and ex!!!t !ecuri!y !!!!
Join a CTF online and learn !!!m !nywhe!e !!!!
Solve tasks win points and h!!! f!n in !TF!


In [13]:
nkey = list(nkey)
pe(2,29,"w",nkey,cc,lines_encrypted)

c5ac0412a6b33cbc6d2565f212e0969b34267700270f8c2f01799eac590000e187001f0b5484fe00c46200000000
CTFs are fun games to test ha!!in! skil!s
They have tasks like web malw!!e !wn cr!pt!
CTFs help you learn how to ha!! a!d sta! s!!!
Work alone or in a team to so!!e !ool p!ob!!!!
CTFs are for both new and exp!!t !ecuri!y !!!!
Join a CTF online and learn f!!m !nywhe!e !!!!
Solve tasks win points and ha!! f!n in !TF!


In [14]:
nkey = list(nkey)
pe(2,30,"a",nkey,cc,lines_encrypted)

c5ac0412a6b33cbc6d2565f212e0969b34267700270f8c2f01799eac59b200e187001f0b5484fe00c46200000000
CTFs are fun games to test hac!in! skil!s
They have tasks like web malwa!e !wn cr!pt!
CTFs help you learn how to hac! a!d sta! s!!!
Work alone or in a team to sol!e !ool p!ob!!!!
CTFs are for both new and expe!t !ecuri!y !!!!
Join a CTF online and learn fr!m !nywhe!e !!!!
Solve tasks win points and hav! f!n in !TF!


In [15]:
nkey = list(nkey)
pe(2,31,"r",nkey,cc,lines_encrypted)

c5ac0412a6b33cbc6d2565f212e0969b34267700270f8c2f01799eac59b276e187001f0b5484fe00c46200000000
CTFs are fun games to test hackin! skil!s
They have tasks like web malware !wn cr!pt!
CTFs help you learn how to hack a!d sta! s!!!
Work alone or in a team to solve !ool p!ob!!!!
CTFs are for both new and expert !ecuri!y !!!!
Join a CTF online and learn from !nywhe!e !!!!
Solve tasks win points and have f!n in !TF!


In [16]:
nkey = list(nkey)
pe(1,34,"g",nkey,cc,lines_encrypted)

c5ac0412a6b33cbc6d2565f212e0969b34267700270f8c2f01799eac59b276e187ad1f0b5484fe00c46200000000
CTFs are fun games to test hacking skil!s
They have tasks like web malware pwn cr!pt!
CTFs help you learn how to hack and sta! s!!!
Work alone or in a team to solve cool p!ob!!!!
CTFs are for both new and expert securi!y !!!!
Join a CTF online and learn from anywhe!e !!!!
Solve tasks win points and have fun in !TF!


In [17]:
nkey = list(nkey)
pe(1,40,"l",nkey,cc,lines_encrypted)

c5ac0412a6b33cbc6d2565f212e0969b34267700270f8c2f01799eac59b276e187ad1f0b5484fe63c46200000000
CTFs are fun games to test hacking skills
They have tasks like web malware pwn crypt!
CTFs help you learn how to hack and stay s!!!
Work alone or in a team to solve cool prob!!!!
CTFs are for both new and expert security !!!!
Join a CTF online and learn from anywhere !!!!
Solve tasks win points and have fun in CTF!


In [18]:
nkey = list(nkey)
pe(2,43,"o",nkey,cc,lines_encrypted)

c5ac0412a6b33cbc6d2565f212e0969b34267700270f8c2f01799eac59b276e187ad1f0b5484fe63c4625c000000
CTFs are fun games to test hacking skills
They have tasks like web malware pwn crypto
CTFs help you learn how to hack and stay sa!!
Work alone or in a team to solve cool probl!!!
CTFs are for both new and expert security f!!!
Join a CTF online and learn from anywhere f!!!
Solve tasks win points and have fun in CTFs


In [19]:
nkey = list(nkey)
pe(4,44,"e",nkey,cc,lines_encrypted)

c5ac0412a6b33cbc6d2565f212e0969b34267700270f8c2f01799eac59b276e187ad1f0b5484fe63c4625c320000
CTFs are fun games to test hacking skills
They have tasks like web malware pwn crypto
CTFs help you learn how to hack and stay saf!
Work alone or in a team to solve cool proble!!
CTFs are for both new and expert security fa!!
Join a CTF online and learn from anywhere fa!!
Solve tasks win points and have fun in CTFs


In [20]:
nkey = list(nkey)
pe(4,45,"m",nkey,cc,lines_encrypted)

c5ac0412a6b33cbc6d2565f212e0969b34267700270f8c2f01799eac59b276e187ad1f0b5484fe63c4625c324c00
CTFs are fun games to test hacking skills
They have tasks like web malware pwn crypto
CTFs help you learn how to hack and stay safe
Work alone or in a team to solve cool problem!
CTFs are for both new and expert security fan!
Join a CTF online and learn from anywhere fas!
Solve tasks win points and have fun in CTFs


In [21]:
nkey = list(nkey)
pe(4,46,"s",nkey,cc,lines_encrypted)

c5ac0412a6b33cbc6d2565f212e0969b34267700270f8c2f01799eac59b276e187ad1f0b5484fe63c4625c324cf0
CTFs are fun games to test hacking skills
They have tasks like web malware pwn crypto
CTFs help you learn how to hack and stay safe
Work alone or in a team to solve cool problems
CTFs are for both new and expert security fans
Join a CTF online and learn from anywhere fast
Solve tasks win points and have fun in CTFs


In [22]:
nkey = ''.join(nkey)
print(bytes.fromhex(hexxor(nkey,flag_encrypted)).decode('utf-8'))

DEFENSYS{C_5__YBE*R10 0%0SE6C<=U4RI-T9Yqy8mp}
